In [1]:
import pyspark
from pyspark.sql import SparkSession

# instantiate a Spark session, an object that we use to interact with Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
# get the green data
df_green = spark.read.parquet('data/parquet/green/*/*')

In [3]:
df_green.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-22 13:18:32|  2020-01-22 13:45:58|                 N|         1|         244|          41|              1|         5.22|       22.0|  0.0|    0.

**We will be implementing the following SQL query:**
```bash
    SELECT 
        date_trunc('hour', lpep_pickup_datetime) AS hour, 
        PULocationID AS zone,

        SUM(total_amount) AS amount,
        COUNT(1) AS number_records
    FROM
        green
    WHERE
        lpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY
        1, 2
```

**First we load in the green data into a Spark RDD**

In [4]:
# convert DataFrame into an RDD with select columns
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd  # this is the underlying RDD of our current DataFrame

In [5]:
# "take" (look at) the first 5 rows in the rdd
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 17, 54, 10), PULocationID=236, total_amount=12.25),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 10, 23, 37), PULocationID=166, total_amount=5.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 21, 14, 25, 16), PULocationID=152, total_amount=17.46),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 9, 46), PULocationID=51, total_amount=22.01)]

**Above, note that the RDD is made up of `Row` objects are used to build DataFrames in Spark**

### 5 Basic Operations on RDDs: `filter`, `map`, `reduceByKey`, `map`, `toDF`
- `.filter()` = we don’t want outliers
- `.map()` = generates intermediate results better suited for aggregation
- `.reduceByKey()` = merges the values for each key
- `.map()` = unwraps the rows
- `.toDF()` = returns the rows to a DataFrame format properly

In [6]:
# don’t want outliers and only need trips from 1/1/2020, so we create a filter
# i.e., implement a WHERE clause
# .filter([define a function])

# Lambda function = a small anonymous function.
#   - can take any number of arguments, but can only have 1 expression
#   - i.e., `lambda arguments : expression`
#   - ex: Add 10 to argument a, and return the result: 
#     - x = lambda a : a + 10
#     - print(x(5))

# return the first row of the filtered data
rdd.filter(lambda row: True).take(1)
# rdd.filter(lambda row: False).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8)]

In [7]:
# more lambda testing

# This returns even numbers.
# See https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.RDD.filter.html
#   and https://cumsum.wordpress.com/2019/10/12/typeerror-missing-1-required-positional-argument-y/
sc = pyspark.SparkContext.getOrCreate()
rdd = sc.parallelize(c=[1, 2, 3, 4, 5])
rdd.filter(lambda x: x % 2 == 0).collect()

[2, 4]

In [8]:
# remake the RDD
# convert DataFrame into an RDD with select columns
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd  # this is the underlying RDD of our current DataFrame

In [9]:
# we want to filter a Timestamp
from datetime import datetime

start = datetime(year=2020, month=1, day=1)

In [10]:
# filter based on this start date, get the first 3 rows (WHERE clause)
rdd.filter(lambda row: row.lpep_pickup_datetime >= start).take(3)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 17, 54, 10), PULocationID=236, total_amount=12.25),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 10, 23, 37), PULocationID=166, total_amount=5.3)]

In [11]:
# put the above logic into a function because lambdas get messy quite fast
def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

# use the function
rdd.filter(filter_outliers).take(3)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 17, 54, 10), PULocationID=236, total_amount=12.25),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 10, 23, 37), PULocationID=166, total_amount=5.3)]

**To implement the equivalent of GROUP BY, we need the `.map()` function, which when applied a *transformation to every `Row`* returns a *transformed RDD***
- `.filter()` *alone* returns `True` or `False`
- `.map()` is applied to every element of the RDD
    - In our case, it recieves a `Row` object as input and outputs something else

**We create a function to use withing `.map()` that transforms a row by creating a *tuple* composed of a key and a value**
- The key = a tuple of `hour` and `zone`, the same 2 columns of the GROUP BY
- The value is a tuple of `amount` and `number_of_records`, the same 2 columns that are returned by the SQL query above

In [12]:
# apply .map() to implement SUM(total_amount) AS amount and COUNT(1) AS number_records
# In order to be able to GROUP BY, output of .map() must be something in the format (key, value)
#   - where key = a composite key [hour zone] and value will be [amount, number_of_trips]:
def prepare_for_grouping(row):
    # .replace() returns a new DataFrame replacing a value with another value
    # we replace every value more granula than day EXCEPT hour with value 0
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    # create the key tuple
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)
    
    return (key, value)

In [13]:
# apply our new function to GROUP BY
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(5)

[((datetime.datetime(2020, 1, 22, 13, 0), 244), (22.8, 1)),
 ((datetime.datetime(2020, 1, 23, 17, 0), 236), (12.25, 1)),
 ((datetime.datetime(2020, 1, 19, 10, 0), 166), (5.3, 1)),
 ((datetime.datetime(2020, 1, 21, 14, 0), 152), (17.46, 1)),
 ((datetime.datetime(2020, 1, 7, 9, 0), 51), (22.01, 1))]

**Next, we will aggregate this RDD transformed RDD by the key (combine values with the same key into a single record)**
- **i.e., perform a REDUCE operation**

**We use `pyspark.RDD.reduceByKey()` to merge the values for each key using an associative and commutative REDUCE function**
- i.e., it takes elements with the same key and merges them together
- Where `left_value` and `right_value` are tuple of `amount` and `number_of_records`

In [14]:
# do the aggregation/REDUCE function/operation on `amount` and `number_of_records`
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    output_amount = left_amount + right_amount
    output_count = left_count + right_count

    return (output_amount, output_count)

In [17]:
# apply the REDUCE operation to our RDD
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue)  # perform a REDUCE

In [18]:
df_result.take(5)

[((datetime.datetime(2020, 1, 19, 10, 0), 166), (969.2699999999993, 55)),
 ((datetime.datetime(2020, 1, 24, 17, 0), 18), (26.26, 1)),
 ((datetime.datetime(2020, 1, 21, 13, 0), 33), (535.8900000000001, 26)),
 ((datetime.datetime(2020, 1, 15, 21, 0), 42), (336.1400000000001, 23)),
 ((datetime.datetime(2020, 1, 4, 12, 0), 116), (119.85, 8))]

**This nested structure seen right above (nested tuples/`Row`'s) is not too intuitive nor easy to manipulate, so we want to turn it back into a DataFrame**

**To do so, we apply another `.map()` function to apply a function to each element (`Row`) of the RDD and transform the `Row`'s into desired DataFrame columns (i.e., unwrap this structure)**

**We first use a `namedtuple` to preserve columns names**

In [35]:
from collections import namedtuple

# returns a new subclass of tuple with named fields
RevenueRow = namedtuple(typename='RevenueRow', field_names=['hour', 'zone', 'revenue', 'count'])

In [20]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

**To return to a DataFrame properly, we want to specify the schema**

In [36]:
# df_result = rdd \
#     .filter(filter_outliers) \
#     .map(prepare_for_grouping) \
#     .reduceByKey(calculate_revenue) \
#     .map(unwrap) \
#     .toDF()  # return the RDD as a DataFrame

In [37]:
# df_result.schema

In [21]:
from pyspark.sql import types

result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [22]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)  # return the RDD as a DataFrame

In [38]:
df_result.show(5)

+-------------------+----+-----------------+-----+
|               hour|zone|          revenue|count|
+-------------------+----+-----------------+-----+
|2020-01-19 10:00:00| 166|969.2699999999993|   55|
|2020-01-24 17:00:00|  18|            26.26|    1|
|2020-01-21 13:00:00|  33|535.8900000000001|   26|
|2020-01-15 21:00:00|  42|336.1400000000001|   23|
|2020-01-04 12:00:00| 116|           119.85|    8|
+-------------------+----+-----------------+-----+
only showing top 5 rows



In [24]:
# write the resulting DataFrame to a temp parquet file
df_result.write.parquet('tmp/green-revenue')

In [25]:
!dir tmp

 Volume in drive C has no label.
 Volume Serial Number is 08A3-CF2D

 Directory of C:\Users\nimz\Documents\de_zoomcamp\week5_batch_processing\tmp

05/11/2023  07:54 PM    <DIR>          .
05/11/2023  07:54 PM    <DIR>          ..
05/11/2023  07:54 PM    <DIR>          green-revenue
05/11/2023  06:42 PM    <DIR>          revenue-zones
               0 File(s)              0 bytes
               4 Dir(s)  368,424,480,768 bytes free
